In [1]:
from tqdm import tqdm
from cooking_bot.encoders import get_sentence_embedding
import json
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
from cooking_bot import REPO_PATH
import os
os.chdir(REPO_PATH)

df = pd.read_csv("datasets/new_clean.csv", index_col=0)
df

,text,queries,time,difficulty
0,How to make chicken curry?,2,0,0
1,Easy breakfast ideas for busy mornings,3,0,1
2,Find a recipe that looks visually stunning,0,0,0
3,How to make a traditional apple pie?,2,0,0
4,How to prepare a classic chicken alfredo?,2,0,0
...,...,...,...,...
804,How to make a simple vegetable stir-fry,2,0,0
805,I want a dessert that looks beautiful,0,0,0
806,Looking for a visually appealing appetizer,0,0,0
807,"I'm looking for an easy, visually appealing snack",0,0,0


In [2]:
embeddings = np.array([get_sentence_embedding(text) for text in tqdm(df.text)])
labels = df.queries

100%|██████████| 809/809 [00:13<00:00, 58.16it/s] 


[0 1 2 3]


In [3]:

X_train, X_test, y_train, y_test = train_test_split(
    embeddings, labels, test_size=0.2, random_state=42
)
# Initialize additional classifiers
svm_classifier = SVC()

# Train additional classifiers
svm_classifier.fit(X_train, y_train)

# Predict using Gradient Boosting

# Predict using SVM
svm_predictions = svm_classifier.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_predictions)
print(f"Support Vector Machine Accuracy: {svm_accuracy}")

Support Vector Machine Accuracy: 0.9753086419753086


In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

tokenizer = AutoTokenizer.from_pretrained("NOVA-vision-language/task-intent-detector")
config = AutoConfig.from_pretrained("NOVA-vision-language/task-intent-detector", output_hidden_states = True)
model = AutoModelForSequenceClassification.from_pretrained("NOVA-vision-language/task-intent-detector",config = config)

In [28]:
def get_length(text):
    a = tokenizer(text, return_tensors="pt")
    return a["input_ids"].shape[1]

max(get_length(t) for t in df.text)

17

In [30]:
def get_hidden_state(text):

    input = tokenizer(text, return_tensors="pt", padding="max_length", max_length=30)

    return model(**input, ).hidden_states[-1][0]

tensor([[-0.1147,  0.7359, -0.1286,  ..., -2.2303, -0.0357,  0.4332],
        [-0.2638,  1.0042, -0.1725,  ..., -2.1180, -0.0436,  0.5972],
        [-0.3286,  1.0986, -0.2611,  ..., -2.1533,  0.0052,  0.5677],
        ...,
        [-0.1160,  0.7341, -0.1286,  ..., -2.2326, -0.0353,  0.4339],
        [-0.1160,  0.7341, -0.1286,  ..., -2.2326, -0.0353,  0.4339],
        [-0.1160,  0.7341, -0.1286,  ..., -2.2326, -0.0353,  0.4339]],
       grad_fn=<SelectBackward0>)